This notebook uses quantized version of Llama2 13B-chat to de-identify private information of i2b2 data.

#**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**Step 1: Install All the Required Packages**

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 11.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 65.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 33.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 9.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-31sa1klv/overlay/local/bin
    changing mode of /tmp/pip-build-env-31sa1klv/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-31sa1klv/overlay/

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

#**Step 3: Download the Model**

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Create a Prompt Template**

In [ ]:
# from os import system
# user_prompt = '''Record date: 2074-09-21
# MICHAEL NOLAND
# FWH Unit No:  233-87-20
# MINERAL SPRINGS CLINIC
# ADULT MEDICINE
# Date:  September 21, 2074
# 46-year-old male presenting for follow up of his blood pressure. Since his last visit a month ago, he has been exercising, going out for walks every other day, compliant with medications. No new complaints for today.
# '''


# system_prompt = ''' Please anonymize the following clinical note. Replace all the following information with the term "########": Redact any strings that might be a name or acronym or initials, patients' names, doctors' names, the names of the M.D. or Dr., redact any pager names, medical staff names, redact any strings that might be a location or address, such as "3970 Longview Drive", redact any strings that look like "something years old" or "age 37", redact any dates(like 2081-9-29) and IDs and record dates, redact clinic and hospital names, redact professions such as "manager", redact any contact information. '''

# prompt_template=f'''<s>[INST] <<SYS>>\n\n{system_prompt}<</SYS>>\n\n{user_prompt}[/INST]'''

In [ ]:
# prompt = "Write a linear regression in python"
# prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

# USER: {prompt}

# ASSISTANT:
# '''

#**Step 6: Preparing data**

In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
directory = "/content/drive/My Drive/GE Capstone/Data/i2b2/testing-PHI-Gold-fixed"

from bs4 import BeautifulSoup
# process dataset, parse each xml file into a text and a tag dict
list_of_text_contents = []
list_of_files = []
list_of_tags = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        # print(os.path.basename(os.path.normpath(f))[:-4])
        list_of_files.append(os.path.basename(os.path.normpath(f))[:-4])
        with open(f) as fp:
            tag_list = []
            soup = BeautifulSoup(fp, features="xml")
            text = soup.find('TEXT')
            text_content = text.contents[0]
            list_of_text_contents.append(text_content)
            tags = soup.find('TAGS')
            for item in tags.children:
              try:
                newdict = {"text":item['text'], "type":item["TYPE"]}
                tag_list.append(newdict)
              except:
                pass

            list_of_tags.append(tag_list)


In [ ]:
# get the indices of the selected 100 files
index = [] #index list
for idx, each in enumerate(list_of_text_contents):
	if len(each) < 2278:
		index.append(idx)
len(index)

100

In [ ]:
# generate sample lists
sample_list_of_files = [list_of_files[i]for i in index]
sample_list_of_text_contents = [list_of_text_contents[i]for i in index]
sample_list_of_tags = [list_of_tags[i]for i in index]

len(sample_list_of_text_contents)
len(sample_list_of_tags)

100

In [ ]:
record_df = pd.DataFrame(sample_list_of_text_contents, columns=["record"])
record_df

,record
0,\n\n\nRecord date: 2086-05-25\n\n ...
1,\n\n\nRecord date: 2092-03-26\n\n\n\nReason fo...
2,\n\n\nRecord date: 2083-07-20\n\n ...
3,\n\n\nRecord date: 2125-07-24\n\nRoutine F/U\n...
4,\n\n\nRecord date: 2078-05-31\n\nVASCULAR SURG...
...,...
95,\n\n\nRecord date: 2096-02-13\n\nSaint John Va...
96,\n\n\nRecord date: 2093-04-30\n\n\n\nReason fo...
97,\n\n\nRecord date: 2095-08-06\n\nOncology COMP...
98,\n\n\nRecord date: 2092-05-24\n\n\n\n\nBAYONNE...


Split report text into sentences of 100 tokens

In [ ]:
### split_text function
def split_text(text, slice_length):
  import re
  text_tokens = re.split(r'[\s\t\n]+', text)
  text_tokens = [x for x in text_tokens if x != '']
  n = len(text_tokens)
  output_ls = []
  num_pieces = n//slice_length + 1

  start_pointer = 0
  end_pointer = slice_length

  for i in range (num_pieces):
    text_slice = text_tokens[start_pointer:end_pointer]
    output_ls.append(" ".join(text_slice))
    start_pointer = end_pointer
    end_pointer = end_pointer + slice_length
  return output_ls

# Split each row of report into list of sentences in 100 tokens
split_df = record_df["record"].map(lambda row: split_text(row, 100))
split_df = pd.DataFrame(split_df).reset_index()
split_df = split_df[[split_df.columns[-1]]]

# Explode the list of sentences in each row into sentence by row
# split_df_row_sep = split_df.explode(split_df.columns[-1]).reset_index()
# split_df_row_sep = split_df_row_sep[[split_df_row_sep.columns[-1]]]
#split_df_row_sep.to_csv('/content/drive/My Drive/GE Capstone/Data/ASRS data/splitted_report_sentbyrow.csv', index=False)

In [ ]:
split_df

,record
0,[Record date: 2086-05-25 FCH EMERGENCY DEPT VI...
1,[Record date: 2092-03-26 Reason for Visit Here...
2,[Record date: 2083-07-20 SILVER RIDGE EMERGENC...
3,[Record date: 2125-07-24 Routine F/U =========...
4,[Record date: 2078-05-31 VASCULAR SURGERY H&P ...
...,...
95,[Record date: 2096-02-13 Saint John Valley Hea...
96,[Record date: 2093-04-30 Reason for Visit Here...
97,[Record date: 2095-08-06 Oncology COMPLETION N...
98,[Record date: 2092-05-24 BAYONNE MEDICAL CENTE...


#**Step 6: Generating the Response**

In [ ]:
from IPython.core import error
import pandas as pd

system_prompt = '''Please anonymize the following clinical note. Replace all the following information with the term '########':
Redact any strings that might be a name or acronym or initials, patients' names, doctors' names, the names of the M.D. or Dr.,
 redact any pager names, medical staff names, redact any strings that might be a location or address, such as '3970 Longview Drive',
  redact any strings that look like 'something years old' or 'age 37', redact any dates (like 2081-9-29) and IDs and record dates,
   redact clinic and hospital names, redact professions such as 'manager', redact any contact information:'''

def generate_prompts(system_prompt, user_prompt):
  prompt_template=f'''<s>[INST] <<SYS>>\n\n{system_prompt}<</SYS>>\n\n{user_prompt}[/INST]'''
  return prompt_template

def process_with_llama2(prompt_template):
    response = lcpp_llm(prompt=prompt_template, max_tokens=1024, temperature=0.5, top_p=0.95,
                        repeat_penalty=1.2, top_k=150, echo=False)
    return response["choices"][0]["text"]

# def count(prompt):
#     words = prompt.split()
#     return len(words)

result_all = []
for row in shukai['record']:
  for input in row:
    user_prompt  = input
    try:
      result = process_with_llama2(generate_prompts(system_prompt, user_prompt))
      #parts = result.split("\n")
      print(result)
      result_all.append(result)
    except Exception as error:
      print(error)
      print('user_prompt')
      print('result')


# result_df = ' '.join(result_df)
result_dict = pd.DataFrame({
    'result_all' : result_all
})
# # After all rows have been processed, save the DataFrame to a CSV
# result_dict.to_csv('/content/drive/My Drive/GE Capstone/Data/i2b2/llama2_result/llama_processed_shukai.csv', index=False)

  Sure! Here's the anonymized clinical note:

Record date: ############# Problems HTN History of Present Illness Lovely ### year old woman with a history of hypertension to establish primary care. She is feeling generally well, though feels that her bp med (hyzaar) makes her feel weak and lightheaded. Would like to change meds. Otherwise, she has no complaints. Past Medical History See above Medications DYAZIDE (TRIAMTERENE 37.5 MG/HCTZ


Llama.generate: prefix-match hit


  Sure! Here's the anonymized clinical note:

Patient is a retired health policy consultant who emigrated from Hungary in the 2060s. No significant medical history or medications reported. Vital signs within normal limits: Blood Pressure 122/70, Left Arm Weight 126 pounds. Physical exam reveals a well-appearing patient with no appreciable lesions on skin, HEENT, PERRLA, and OP benign. Neck is supple without lymph


Llama.generate: prefix-match hit


  Sure! Here is the anonymized clinical note:

Redacted Patient Information:

Chief Complaint: Healthy woman seeking to change her blood pressure medication.

History of Present Illness: Denies any guarding or rebound tenderness, normal S1 and S2 abdomen soft with no neurological deficits. No history of recent surgeries or injuries. Patient reports that her pap/mammo are up to date as of 11/89 at an unnamed medical center.

Past Medical History: None mentioned.

Medications: Currently taking a trial of dyazide, with plans for follow


...

  Sure! Here is the anonymized clinical note with all personal information replaced with '########':

Record date: #######-###-### CHIEF COMPLAINT: PER DR VARNEY HISTORY OF PRESENT ILLNESS: s/p CABG 2 weeks ago; doing well without Sx CP/SOB; was depressed last week but now feeling


Llama.generate: prefix-match hit


  Sure, here is the anonymized clinical note:

Patient: ############## DOB: ####-####-####
Age: ##### years old Gender: #####

Chief Complaint: None

History of Present Illness: No back, shoulder, limb
Requested tokens (529) exceed context window of 512
user_prompt
result


Llama.generate: prefix-match hit


  Sure! Here is the anonymized version of the clinical note:

### Patient Information ###

* Age: ###### years old
* Gender: ######
* Contact information: ######

### Chief Complaint ###

*##### symptoms since ####, with worsening over the past few days.#######

### History of Present Illness ###

* The patient presented to the clinic with a history of ######, which had been present for approximately ###### years. The patient reported experiencing ###### and ###### symptoms, with worsening over the past few days.

### Review of Systems ###

* General: No relevant information provided.
* Constitutional: No relevant information provided.
* Cardiovascular: No relevant information provided.
* Respiratory: No relevant information provided.
* Gastrointestinal: No relevant information provided.
* Genitourinary: No relevant information provided.
* Musculoskeletal: No relevant information provided.
* Neurological: No relevant information provided.
* Psychiatric: No relevant information provided